# 你的第一个实验
### 请阅读本节。这对你做好准备非常有价值，即使阅读起来较长，但这些内容很重要。

## 你的第一个 Frontier LLM 项目

让我们在几分钟内构建一个有用的 LLM 解决方案。

在本课程结束时，你将构建一个具有 7 个代理（Agent）的自主智能体 AI 解决方案，这些代理将协同解决一个业务问题。一切都会水到渠成！我们将从更小的东西开始...

我们的目标是编写一种新型的网页浏览器。给它一个 URL，它将返回一个摘要。

开始之前，你应该已经完成了 [PC](../SETUP-PC.md) 或 [Mac](../SETUP-mac.md) 的设置，并且最好是在项目根目录内启动了此 Jupyter Lab，并激活了你的环境。

## 如果你是 Jupyter Lab 新手

欢迎来到数据科学实验的精彩世界！一旦你使用过 Jupyter Lab，你会想知道之前没有它你是怎么工作的。只需单击带有代码的每个“单元格”（例如此文本正下方的单元格），然后按 Shift+Return 执行该单元格。如果你愿意，可以使用工具栏中的 + 按钮添加一个单元格，打印变量的值，或尝试各种变体。  

我写了一个名为 [Guide to Jupyter](Guide%20to%20Jupyter.ipynb) 的笔记本，以帮助你更熟悉 Jupyter Labs，包括添加 Markdown 注释，使用 `!` 运行 shell 命令，以及使用 `tqdm` 显示进度。

In [ ]:
# 导入库

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# 如果运行此单元格时出错，请前往此文件夹中的故障排除笔记本进行查看！

# 连接到 OpenAI

下一个单元格是加载你的 `.env` 文件中的环境变量并连接到 OpenAI 的地方。

## 如果遇到问题请进行故障排除：

前往此文件夹中的 [故障排除](troubleshooting.ipynb) 笔记本，其中包含逐步代码以识别根本原因并进行修复！

如果你进行了更改，尝试通过 Kernel 菜单 >> Restart Kernel and Clear Outputs of All Cells 重启“内核”（此笔记本背后的 Python 进程）。然后从本笔记本顶部开始重新运行单元格。

担心 API 成本吗？请查看 README 中的说明 - 成本应该非常低，而且你可以在每个环节控制。你也可以使用 Ollama 作为免费替代方案，我们会在第 2 天讨论它。

In [ ]:
# 从名为 .env 的文件中加载环境变量

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# 检查 API 密钥

if not api_key:
    print("未找到 API 密钥 - 请前往此文件夹中的故障排除笔记本进行识别和修复！")
elif not api_key.startswith("sk-proj-"):
    print("找到了 API 密钥，但它不是以 sk-proj- 开头；请检查你是否使用了正确的密钥 - 请参阅故障排除笔记本")
elif api_key.strip() != api_key:
    print("找到了 API 密钥，但它看起来开头或结尾可能包含空格或制表符 - 请删除它们 - 请参阅故障排除笔记本")
else:
    print("API 密钥已找到，目前看起来不错！")


In [ ]:
openai = OpenAI()

# 如果这不起作用，请尝试 Kernel 菜单 >> Restart Kernel and Clear Outputs Of All Cells，然后从本笔记本顶部开始运行单元格。
# 如果仍然不起作用（天哪！），请参阅此文件夹中的故障排除笔记本以获取完整说明

# 让我们快速调用一下 Frontier 模型作为预览！

In [ ]:
# 为了给你一个预览 -- 用这些消息调用 OpenAI 就是如此简单。如有任何问题，请前往故障排除笔记本。

message = "你好，GPT！这是我发给你的第一条消息！你好！"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

## 好的，继续我们的第一个项目

In [ ]:
# 用于表示网页的类

# 一些网站在抓取时需要使用正确的请求头：
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        使用 BeautifulSoup 库从给定的 url 创建此 Website 对象
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "未找到标题"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# 让我们试一个。更改网站并添加 print 语句来跟踪。

ed = Website("https://iloveaws.cn")
print(ed.title)
print(ed.text)

## 提示类型

你可能已经知道这一点 - 但如果不知道，你很快就会非常熟悉它！

像 GPT4o 这样的模型已经被训练成以特定方式接收指令。

它们期望接收：

**一个系统提示（System Prompt）**，告诉模型它正在执行什么任务以及应使用什么语气

**一个用户提示（User Prompt）** -- 模型应该回复的对话启动者

In [ ]:
# 定义我们的系统提示 - 你可以在稍后进行实验，将最后一句话更改为 '以英文的 markdown 格式回复。"

system_prompt = "你是一个分析网站内容并提供简短摘要的助手，忽略可能与导航相关的文本。\
请以 markdown 格式回复。"

In [ ]:
# 一个用于编写请求网站摘要的用户提示的函数：

def user_prompt_for(website):
    user_prompt = f"你正在查看一个标题为 {website.title} 的网站"
    user_prompt += "\n此网站的内容如下；请以 markdown 格式提供此网站的简短摘要。\
如果网站包含新闻或公告，也请一并进行摘要。\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(ed))

## 消息

来自 OpenAI 的 API 希望以特定的结构接收消息。
许多其他 API 也共享此结构：

```
[
    {"role": "system", "content": "系统消息在这里"},
    {"role": "user", "content": "用户消息在这里"}
]

为了给你一个预览，接下来的 2 个单元格进行了一个相当简单的调用 - 我们（暂时！）不会挑战强大的 GPT

In [ ]:
# 一个简单的消息列表

messages = [
    {"role": "system", "content": "你是一个喜欢挖苦人的助手"},
    {"role": "user", "content": "2 + 2 是多少？"}
]

In [ ]:
# 为了给你一个预览 -- 调用 OpenAI 并使用系统和用户消息：

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

## 现在让我们使用函数为 GPT-4o-mini 构建有用的消息

In [ ]:
# 看看这个函数是如何创建上面那种格式的

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
# 试试这个，然后尝试一些其他网站

messages_for(ed)

## 是时候将它们结合起来了 - OpenAI 的 API 非常简单！

In [ ]:
# 现在：调用 OpenAI API。你会非常熟悉这个的！

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://iloveaws.cn")

In [ ]:
# 一个函数用于使用 markdown 在 Jupyter 输出中漂亮地显示摘要

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://iloveaws.cn")

# 让我们尝试更多网站

注意，这仅适用于可以使用这种简单方法抓取的网站。

使用 Javascript 渲染的网站（如 React 应用）将无法显示。请参阅 community-contributions 文件夹，其中有 Selenium 的实现可以解决这个问题。你需要查阅如何安装 Selenium（问问 ChatGPT！）。

此外，使用 CloudFront（及类似服务）保护的网站可能会返回 403 错误 

但许多网站会正常工作！

In [ ]:
display_summary("https://cnn.com")

In [ ]:
display_summary("https://anthropic.com")

In [ ]:
# 步骤 1：创建你的提示

system_prompt = "这里写一些内容"
user_prompt = """
    很多文本
    可以在这里粘贴
"""

# 步骤 2：创建消息列表

messages = [] # 在这里填写

# 步骤 3：调用 OpenAI

response =

# 步骤 4：打印结果

print(

## 对于喜欢网页抓取的人的额外练习

你可能会注意到，如果你尝试 `display_summary("https://openai.com")` - 它不起作用！那是因为 OpenAI 有一个使用 Javascript 的高级网站。有很多方法可以绕过这个问题，其中一些你可能已经熟悉。例如，Selenium 是一个非常流行的框架，它在后台运行浏览器，渲染页面，并允许你查询它。如果你有 Selenium、Playwright 或类似工具的经验，请随时改进 Website 类来使用它们。在 community-contributions 文件夹中，你会找到一个学生提供的 Selenium 解决方案示例（谢谢！）。